## Remember to open terminal and run: conda activate ilab-tensorflow to import shap

In [ ]:
import shap

In [ ]:
import sys
sys.path.append('/explore/nobackup/people/gtamkin/dev/AGB/mpf-model-factories/MultiPathFusion')
from multi_path_fusion.src.utils.data_generator_helpers import load_data_generator

In [ ]:
import numpy as np
import pickle
import tensorflow as tf
from scipy.special import softmax

In [ ]:
def print_feature_importances_shap_values(shap_values, features):
    
    '''
    Prints the feature importances based on SHAP values in an ordered way
    shap_values -> The SHAP values calculated from a shap.Explainer object
    features -> The name of the features, on the order presented to the explainer
    '''
    
    # Calculates the feature importance (mean absolute shap value) for each feature
    importances = []
    for i in range(shap_values.values.shape[1]):
        importances.append(np.mean(np.abs(shap_values.values[:, i])))
        
    # Calculates the normalized version
    importances_norm = softmax(importances)

    # Organize the importances and columns in a dictionary
    feature_importances = {fea: imp for imp, fea in zip(importances, features)}
    feature_importances_norm = {fea: imp for imp, fea in zip(importances_norm, features)}

    # Sorts the dictionary
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}
    feature_importances_norm= {k: v for k, v in sorted(feature_importances_norm.items(), key=lambda item: item[1], reverse = True)}

    # Prints the feature importances
    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f} (softmax = {feature_importances_norm[k]:.4f})")

In [ ]:
def evaluate_regression(y, y_pred):
    
    '''
    Prints the most common evaluation metrics for regression
    '''
    
    mae = MAE(y, y_pred)
    mse = MSE(y, y_pred)
    rmse = mse ** (1/2)
    r2 = R2(y, y_pred)
    
    print('Regression result')
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R2: {r2:.2f}")

In [ ]:
keywordAll = ['ARI', 'CAI', 'CRI550', 'CRI700', 'EVI', 'EVI2', 'fPAR', 'LAI', 'MCTI', 'MSI',
                'NDII', 'NDLI', 'NDNI', 'NDVI', 'NDWI', 'NIRv', 'PRIn', 'PRIw', 'SAVI', 'WBI', 'Albedo']
keyword7 = list()
keyword7.append('PRIw')
keyword7.append('NDVI')
keyword7.append('MCTI')
keyword7.append('CRI550')
keyword7.append('WBI')
keyword7.append('LAI')
keyword7.append('MSI')
keywordAllInt = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


## Restore model() and test-generator() data [or just Explaniner/shap_values

In [ ]:
modelPath = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/12262023/MODELS/Exp_7bands_pickle::502671461260014182.keras'
print('reloading model:', modelPath)
model21 = tf.keras.models.load_model(modelPath)    


## Restore test_generator() data using pickle. The test_generator() was saved cleanly.

In [ ]:
archive_test_generator_id = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/12262023/TRIALS/Exp_7bands_pickle::502671461260014182.keras.test_generator.data'
#archive_test_generator_id = archive_id + "_test_generator.data"
#pickle.dump(test_generator, open(archive_test_generator_id, "wb"))
test_generator_r = pickle.load(open(archive_test_generator_id, "rb"))

## Convert test_generator.file_x_stack to Pandas dataframe and transpose for SHAP API.

In [ ]:
import pandas as pd
df = pd.DataFrame(data=test_generator_r.file_x_stack)
dft = df.transpose()
X = X_test = dft
print(df.shape, X.shape)

# Use KernelExplainer to derive the shap values for ALL 21 bands as explained with first 50 test instances (rows)

In [ ]:
explainer21 = shap.KernelExplainer(model21.predict, X.iloc[:50, :])

In [ ]:
shap_values21_0to50 = explainer21.shap_values(X.iloc[0:50, :], nsamples=500)

In [ ]:
# print according to most in particular bin:  0 < -1.0, 1 < -0.5, 2 < 0, 3 < 0.5, 4 < 1.0
print('bin value 0: [-1.0, -0.5]] = summary_plot(shap_values0to50[0]') 
shap.summary_plot(shap_values21_0to50[0], X.iloc[0:50, :], plot_type="bar", feature_names=keywordAll)

In [ ]:
# print according to most in particular bin:  0 < -1.0, 1 < -0.5, 2 < 0, 3 < 0.5, 4 < 1.0
print('bin value 0: [-1.0, -0.5]] = summary_plot(shap_values0to50[0]') 
shap.summary_plot(shap_values21_0to50, X.iloc[280:330, :], plot_type="bar", feature_names=keywordAll)

In [ ]:
shap.summary_plot(shap_values21_0to50, X, plot_type="bar", feature_names=keywordAll)

# Use ExactExplainer is required for certain plots. Can not support 3D shap.values() but can handle a reduced Explanation object (sort of).

In [ ]:
Explainer21 = shap.Explainer(model21.predict, X.iloc[:50, :])
explanation21 = Explainer21(X.iloc[:50, :])

In [ ]:
print(explanation21.values.shape, explanation21.base_values.shape, explanation21.data.shape)

In [ ]:
#exp = shap.Explanation(explanation21)
#e_shap_values = shap.Explanation(explanation21.values[0], base_values=explanation21.base_values, data=explanation21.data)
e_shap_values = shap.Explanation(explanation21.values[0], base_values=explanation21.base_values)
e_shap_values.shape

In [ ]:
shap.plots.bar(e_shap_values)

In [ ]:
shap.plots.beeswarm(e_shap_values)

In [ ]:
shap.summary_plot(e_shap_values)

In [ ]:
shap.summary_plot(e_shap_values, plot_type='violin')

In [ ]:
X_test.columns

In [ ]:
# Prints the SHAP feature importances
print_feature_importances_shap_values(e_shap_values, X_test.columns)

In [ ]:
shap.plots.bar(e_shap_values[0])

In [ ]:
print(explanation21.values.shape, explanation21.base_values.shape, explanation21.data.shape)

In [ ]:
print(explanation21.base_values[0])

In [ ]:
print(explainer21.expected_value)

In [ ]:
e_shap_values.values = explanation21.values[0][0]
e_shap_values.base_values = explanation21.base_values[0]
e_shap_values

In [ ]:
print(explainer21.expected_value)

In [ ]:
#shap.plots.force(e_shap_values, shap_values21_0to50[0])

In [ ]:
#shap.plots.waterfall(e_shap_values, max_display=10, show=True)

# ...Try ExactExplainer to derive the shap values for ALL 21 bands as explained with ALL 1000x1000 test instances (rows)

In [ ]:
# Fits the explainer
explainerAll = shap.Explainer(model21.predict, X_test)

In [ ]:
# Calculates the SHAP values - It takes some time
shap_valuesAll = explainerAll(X_test)

In [ ]:
explainerAll = shap.KernelExplainer(model21.predict, X.iloc[:50, :])

In [ ]:
shap_values21_0to50 = explainer21.shap_values(X.iloc[0:50, :], nsamples=500)

In [ ]:
shap.plots.bar(shap_values,feature_names=keywordAll)

In [ ]:
# See above.  In this call, we use the first 50 rows of the test-generator dataset as the background dataset.  Should be configurable.  QUESTION: Should this be train_generator instead (perhaps with K-means)
explainer = shap.KernelExplainer(model.predict, X.iloc[:50, :])
print(explainer)

## Save/restore KernelExplainer data using pickle().  Explainer saved cleanly.

In [ ]:
archive_explainer_id = archive_id + model_config.get("model_name") + "_kernel.explainer"
pickle.dump(explainer, open(archive_explainer_id, "wb"))
explainer_r = pickle.load(open(archive_explainer_id, "rb"))

In [ ]:
differences = DeepDiff(explainer, explainer_r)
pprint(differences)

In [ ]:
print('KernelExplainer I/O: ')
print('\n Explainer -> features: ')
print('data_feature_names', explainer.data_feature_names)
print('len(expected_value)', len(explainer.expected_value))
print('expected_value', explainer.expected_value)

print('\n Explainer_r -> features: ')
print('data_feature_names', explainer_r.data_feature_names)
print('len(expected_value)', len(explainer_r.expected_value))
print('expected_value', explainer_r.expected_value)


In [ ]:
archive_explainer_id

In [ ]:
# Get the shap values for the first 50 rows in the test-generator dataset - based on 500 samples each (should be configurable)
shap_values0to50 = explainer.shap_values(X.iloc[0:50, :], nsamples=500)


In [ ]:
# sshap_values0to50.shape() = (50, 7). - first 50 rows by 7 band columns
# there are 5 rows of shape values because binning results in 5 classifications
#print(len(shap_values0to50), shap_values0to50[0].shape , shap_values0to50[4][0].shape)
lastShapRow = shap_values0to50[0][0]
print(lastShapRow.shape, lastShapRow[0].max(), lastShapRow)
sum(explainer_r.expected_value[0] - lastShapRow)

In [ ]:
bandLen = 20

bandOccurenceArr = np.zeros(bandLen)
print(bandOccurenceArr)
bandMaxArr = np.zeros(bandLen)
bandMinArr = np.zeros(bandLen)
bandMeanArr = np.zeros(bandLen)


In [ ]:
explainer.explain_row(0)

In [ ]:
#shap_values0to50 = explainer.shap_values(X.iloc[0:50, :], nsamples=500)
shap_values_explanation = explainer(X.iloc[0:50, :])

In [ ]:
#print(shap_values_explanation[0], shap_values_explanation[0].values[0][0],  shap_values_explanation[0].data[0])
print(shap_values_explanation[0])

In [ ]:
base_delta = (shap_values_explanation.base_values[0] - shap_values_explanation.values[0])
pprint(base_delta)
col_totals = [ sum(x) for x in zip(*base_delta) ]
pprint(col_totals)

In [ ]:
print('expected_value', explainer_r.expected_value)
print(' - expected_value', explainer_r.expected_value)


In [ ]:
shap_values_explanation

In [ ]:
print(X.iloc[0:, :])

In [ ]:
archive_shap_values_explanation_id = archive_id + model_config.get("model_name") + "_shap.explanation"
pickle.dump(shap_values_explanation, open(archive_shap_values_explanation_id, "wb"))
archive_shap_values_explanation_id_r = pickle.load(open(archive_shap_values_explanation_id, "rb"))

In [ ]:
differences = DeepDiff(shap_values_explanation, shap_values_explanation_id_r)
pprint(differences)

## Repeat logic using X_train instead of parts of X_test

In [ ]:
import pandas as pd 
df = pd.DataFrame(data=train_generator.file_x_stack)
dft = df.transpose()
X_train = dft
print(df.shape, X_train.shape)

In [ ]:
# See above.  In this call, we use the first 50 rows of the test-generator dataset as the background dataset.  Should be configurable.
explainer_X_train = shap.KernelExplainer(model.predict, X_train)
#explainer_X_train = shap.KernelExplainer(model.predict, X_train.iloc[:50, :])
print(explainer_X_train)

In [ ]:
shap_values0to50_X_train = explainer.shap_values(X.iloc[0:50, :], nsamples=500)


In [ ]:
print(model.summary())

In [ ]:
from keras.utils.vis_utils import plot_model  
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
def print_accuracy(f):
    print(
        "Root mean squared test error = {}".format(
            np.sqrt(np.mean((f(X_test) - y_test) ** 2))
        )
    )
    time.sleep(0.5)  # to let the print get out before any progress bars


shap.initjs()

In [ ]:
import pandas as pd
df = pd.DataFrame(data=test_generator.file_x_stack)
df

In [ ]:
dft = df.transpose()
dft.iloc[:, :]

In [ ]:
X = dft
print(X.shape)
print(X.shape[0])
print(X.shape[1])

In [ ]:
print(X.shape[1])
count = 0
for i in range(X.shape[1]):
    count = count + 1

print(count)
#print(X[:,count])

In [ ]:
#print([X[:, i] for i in range(X.shape[1])])
print(X.shape)
print(X.iloc[:, :])
print(X.iloc[:5, :])
print(X.iloc[0, 0])
print(X.iloc[3, 6])
print(X.iloc[:3, :])

In [ ]:
def my_predict2(X):
    return model.predict([X[:, i] for i in range(X.shape[1])]).flatten()

In [ ]:
import shap

# load your data here, e.g. X and y
# create and fit your model here

# load JS visualization code to notebook
shap.initjs()
print(model.summary())

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.KernelExplainer(model.predict, X.iloc[:50, :])
#explainer = shap.KernelExplainer(my_predict2, X.iloc[:2, :])

In [ ]:
#print(X.iloc[:50, :])
shap_values = explainer.shap_values(X.iloc[299, :], nsamples=500)
shap_values2 = explainer.shap_values(X.iloc[299, :], nsamples=500)
print('shap_values: '+ str(len(shap_values)))
print(shap_values)
print('shap_values2: '+ str(len(shap_values2)))
print(shap_values2)

In [ ]:
X.iloc[299, :]

In [ ]:
shap_values[0].tofile('./mlruns/shap1.shap')

In [ ]:
shap_values0 = np.fromfile('./mlruns/shap1.shap')

In [ ]:
print(train_generator.file_x_stack.shape, train_generator.file_x_stack[0].shape)

In [ ]:
print(shap_values[0], sum(shap_values[0]))
print(shap_values0, sum(shap_values0))

In [ ]:
print(explainer.fx, sum(explainer.fx))
print(explainer.expected_value, sum(explainer.expected_value))
diff = explainer.fx - explainer.expected_value
print(diff, sum(diff))

In [ ]:
padding = 3
bandList = data_generator_config['branch_inputs'][0]['branch_files'][0]['bands']
bandList

In [ ]:
bandRootDir = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/12122023-7b/BANDS'
hyperspectralIndicesFile = data_generator_config['branch_inputs'][0]['branch_files'][0]['mlbs_year_filepath'] 
print(hyperspectralIndicesFile)
hpath, hname = hyperspectralIndicesFile.split('/',1)
hprefix, hsuffix = hname.split('.',1)
print(bandRootDir, hpath, hprefix)
newPath = os.path.join(bandRootDir, hpath)
print(newPath)
if (not os.path.exists(newPath)): os.makedirs(newPath)

# Write out bands
for i in range(len(bandList)):
    newPathFile = os.path.join(newPath, hprefix+'_band'+str(bandList[i]).zfill(padding)+'.band')
    print(i, str(bandList[i]).zfill(padding), newPathFile, train_generator.file_x_stack[i].dtype, train_generator.file_x_stack[i].shape, train_generator.file_x_stack[i].min())
    train_generator.file_x_stack[i].tofile(newPathFile)

In [ ]:
# Read in bands
for i in range(len(bandList)):
    existingBandFile = os.path.join(newPath, hprefix+'_band'+str(bandList[i]).zfill(padding)+'.band')
    bandValues = np.fromfile(existingBandFile)
    print(existingBandFile, bandValues.dtype, bandValues.shape, bandValues.min())

In [ ]:
train_generator.file_x_stack[0].max()

In [ ]:
train_generator.file_x_stack[1].max()

In [ ]:
from tempfile import TemporaryFile
outfile = TemporaryFile()

In [ ]:
band018 = train_generator.file_x_stack[0]
band014 = train_generator.file_x_stack[1]
band009 = train_generator.file_x_stack[2]
band003 = train_generator.file_x_stack[3]
band020 = train_generator.file_x_stack[4]
band008 = train_generator.file_x_stack[5]
band010 = train_generator.file_x_stack[6]
print(band018, band018.dtype, band018.shape)

In [ ]:
np.savez(outfile, band018=band018, band014=band014, band009=band009, band003=band003, band020=band020, band008=band008, band010=band010)

In [ ]:
_ = outfile.seek(0)
print(outfile)

In [ ]:
npzfile = np.load(outfile)

In [ ]:
npzfile.files

In [ ]:
print(band018, band018.dtype, band018.shape, band018.max())
print(npzfile['band018'], npzfile['band018'].dtype, npzfile['band018'].shape, npzfile['band018'].max())

In [ ]:
from tempfile import TemporaryFile
band018file = TemporaryFile()

In [ ]:
np.save(band018file, train_generator.file_x_stack[0])

In [ ]:
_ = band018file.seek(0)
b18 = np.load(band018file)
print(b18, b18.dtype, b18.shape, b18.max())

In [ ]:
explainer.expected_value

In [ ]:
explainer.expected_value[0]

In [ ]:
#1    Index Name=ARI
#2    Index Name=CAI
#3    Index Name=CRI550
#4    Index Name=CRI700
#5    Index Name=EVI
#6    Index Name=EVI2
#7    Index Name=fPAR
#8    Index Name=LAI
#9    Index Name=MCTI
#10    Index Name=MSI
#11   Index Name=NDII
#12    Index Name=NDLI
#13    Index Name=NDNI
#14    Index Name=NDVI
#15    Index Name=NDWI
#16    Index Name=NIRv
#17    Index Name=PRIn
#18    Index Name=PRIw
#19    Index Name=SAVI
#20    Index Name=WBI
#21    Index Name=Albedo

#        "bands": [18, 14, 9, 3, 20, 8, 10]}]
    
keyword = list()
keyword.append('PRIw')
keyword.append('NDVI')
keyword.append('MCTI')
keyword.append('CRI550')
keyword.append('WBI')
keyword.append('LAI')
keyword.append('MSI')
print(keyword)

In [ ]:
x = 0
while x < 5:
    diff = explainer.expected_value[x] - shap_values[x]
    print("explainer.expected_value[x] where x = " + str(x))
    print(explainer.expected_value[x], shap_values[x], diff, 
      diff.min(), diff.max())
    x = x + 1
print(x)
print(explainer.expected_value, sum(explainer.expected_value))
print(explainer.fx, sum(explainer.fx))

In [ ]:
print(X.shape)

In [ ]:
# rather than use the whole training set to estimate expected values, we summarize with
# a set of weighted kmeans, each weighted by the number of points they represent.
#X_test_kmeans_summary = shap.kmeans(X, 1000)
#print(X_test_kmeans_summary[0])

In [ ]:
#print(X_test_kmeans_summary[0])

In [ ]:
print('shap_values: ' + str(len(shap_values)), shap_values)
print('shap_values2: '+ str(len(shap_values2)), shap_values2)
shap_values999 = explainer.shap_values(X.iloc[999, :], nsamples=500)
print('shap_values999: '+ str(len(shap_values999)), shap_values999)
shap_values999b = explainer.shap_values(X.iloc[999, :], nsamples=500)
print('shap_values999b: '+ str(len(shap_values999b)), shap_values999b)

In [ ]:
#shap.summary_plot(shap_values, X.iloc[0, :], plot_type="bar", feature_names=keyword)


In [ ]:
# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
#shap.force_plot(explainer.expected_value[0], shap_values[0], X.iloc[299, :])
shap.force_plot(explainer.expected_value[0], shap_values[0], keyword)

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1], keyword)

In [ ]:
shap.force_plot(explainer.expected_value[2], shap_values[2], keyword)

In [ ]:
shap.force_plot(explainer.expected_value[3], shap_values[3], keyword)

In [ ]:
shap.force_plot(explainer.expected_value[4], shap_values[4], keyword)

In [ ]:
#shap.force_plot(explainer.expected_value[5], shap_values[5], X.iloc[299, :])

In [ ]:
shap_values50 = explainer.shap_values(X.iloc[280:330, :], nsamples=500)

In [ ]:
shap.force_plot(explainer.expected_value[0], shap_values50[0], X.iloc[280:330, :],feature_names=keyword)

In [ ]:
shap.force_plot(explainer.expected_value[4], shap_values50[4], X.iloc[280:330, :],feature_names=keyword)

In [ ]:
shap.summary_plot(shap_values50[0], X.iloc[280:330, :], plot_type="bar", feature_names=keyword)

In [ ]:
shap.summary_plot(shap_values50[2], X, plot_type="bar", feature_names=keyword)

In [ ]:
shap.summary_plot(shap_values50[4], X.iloc[280:330, :], plot_type="bar", feature_names=keyword)
print('shap_values50[4]: ', shap_values50[4].shape, shap_values50[4].sum(), shap_values50[4].min(), shap_values50[4].max())

In [ ]:
shap_values11 = explainer.shap_values(X.iloc[88888:88899, :], nsamples=500)

In [ ]:
shap.summary_plot(shap_values11[0], X.iloc[280:291, :], plot_type="bar", feature_names=keyword)

In [ ]:
print(shap_values, shap_values50, shap_values11)

In [ ]:
shap_values_abs = np.abs(shap_values)
#shap_values_abs = np.abs(shap_values50)
shap_values_abs_sum = np.sum(shap_values_abs, axis=0)
shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum)
print(shap_values_abs, shap_values_abs_sum, shap_values_abs_sum_argsort)
feature_order = np.argsort(np.sum(np.abs(shap_values), axis=0))
print('\n', feature_order)

In [ ]:
a = [[1, 2, 3, 4, 5, 6, -7], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7]]
nd_a = np.array(a)

#shap_values = nd_a
shap_values_abs = np.abs(shap_values)
#shap_values_abs = np.abs(shap_values50)
shap_values_abs_sum = np.sum(shap_values_abs, axis=0)
shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum)
print("\nshap_values: (shap_values)\n", shap_values, shap_values.dtype, shap_values.shape, "\nshap_values_abs: np.abs(shap_values)\n", shap_values_abs, 
      "\nshap_values_abs_sum: np.sum(shap_values_abs, axis=0)\n", shap_values_abs_sum, "\nshap_values_abs_sum_argsort: np.argsort(shap_values_abs_sum)\n", shap_values_abs_sum_argsort)
feature_order = np.argsort(np.sum(np.abs(shap_values), axis=0))
print('\nfeature_order:', feature_order)

In [ ]:
#shap.values[00,7)
shap00_list = [-0.00537375,  0.01205069,  0.04364897,  0.00142398, -0.00139138, -0.08626259,  0.01085853]
shap00_array = np.array(shap00_list)
print(shap00_array.dtype, shap00_array)

In [ ]:
B = np.array([[-9.,11.,-21.,63.,-252.],[3891.,506.,-1008.,3031.,-12117.],[3891.,576.,-1149.,3451.,-13801.],[3891.,-3891.,7782.,-23345.,93365.],[1024.,-1024.,2049.,-6144.,24572.]])
x = np.abs(B).argmax(axis=0)[0]
print(B, x)

In [ ]:
shap_values_abs = np.abs(shap_values)
#shap_values_abs = np.abs(shap_values50)
shap_values_abs_sum = np.sum(shap_values_abs, axis=0)
shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum)
print(shap_values, shap_values_abs, shap_values_abs_sum, shap_values_abs_sum_argsort)
feature_order = np.argsort(np.sum(np.abs(shap_values), axis=0))
print('\n', feature_order)

In [ ]:
shap.summary_plot(shap_values50[3], X.iloc[280:330, :], plot_type="bar", feature_names=keyword)

In [ ]:
shap.summary_plot(shap_values50, X, plot_type="bar", feature_names=keyword)